In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

### Read Dataset

In [6]:
data_path = "../data/Dataset.xlsx"

In [7]:
df_lahan =  pd.read_excel(data_path, sheet_name='Data Kondisi Lahan', header=0)
df_lahan.head()

,No,Desa/Keluarahan,Suhu,Curah Hujan (mm/thn),Kelembapan Udara (%),Jenis Tanah,Tekstur Tanah,Ph Tanah,Kemiringan Lahan (%),Tinggi Tempat (mdpl)
0,1,Kota Uneng,14-33,622-1425,45-80,Mediteran,Lempung berpasir dan bedebu,6 - 7.5,0 - < 4,0-75
1,2,Nangalimang,14-33,622-1425,45-80,Mediteran,Lempung berpasir dan bedebu,6 - 7.5,0 - < 4,25-75
2,3,Wuring,14-33,622-1425,45-80,Mediteran,Liat berpasir dan lempung bedebu,4.2- 6,26-45,0-500
3,4,Wolomarang,14-33,622-1425,45-80,Mediteran,Liat berpasir dan lempung bedebu,4.2- 6,26-45,0-500
4,5,Tilang,23-33,1173-2277,74-86,"Regosol, Latosol dan Grumosol","Liat, Liat berpasir,Lempung berpasir dan lempu...",4.2- 6,15-39,0-1000


In [8]:
df_profil_lahan = pd.read_excel(data_path, sheet_name='Data Profil Ideal Jenis Tanaman', skiprows=2, header=0)
df_profil_lahan.head()

,No,Jenis Tanaman,Suhu (°C),Curah Hujan (mm/tahun),Kelembapan Udara (%),Jenis Tanah,Tekstur Tanah,Tingkat Kemasaman Tanah/Ph (cm),Kemiringan Tanah (%),Tinggi Tempat/Topografi (mdpl)
0,1,Bawang Merah,10-30,350-800,80-90,"Alluvial, Glei Humus, Latosol","Liat berpasir, liat, liat berdebu, Lempung ber...",6-8,"5,5-16",700-1000
1,2,Cabai Merah,18-27,600-1400,18-30,"Alluvial, Glei Humus, Latosol","Liat berpasir, liat, liat berdebu, Lempung ber...",6-8,"5,5-16",900-1800
2,3,Cabai Rawit,18-27,600-1400,18-30,"Alluvial, Glei Humus, Latosol","Liat berpasir, liat, liat berdebu, Lempung ber...",6-8,"5,5-16",900-1800
3,4,Tomat,18-30,400-800,24-90,"Andosol, Latosol, Regosol","Liat berpasir, liat, liat berdebu, Lempung ber...",6-8,"5,5-8",350-750
4,5,Wortel,16-20,65-80,40-90,"Alluvial, Andosol, Latosol, Regosol","Lempung berliat, lempung liat berpasir, lempun...","5,6-8","5,5-8",1000-1200


### Preprocess Data

In [9]:
def split_string(input_string):
    # Mengganti karakter newline dengan spasi
    cleaned_string = input_string.replace('\n', ' ')

    # Mengubah semua karakter menjadi huruf kecil
    cleaned_string = cleaned_string.lower()

    # Pisahkan string pertama dengan pemisah "dan"
    parts_with_and = cleaned_string.split(" dan ")

    # Buat list kosong untuk menyimpan hasil akhir
    result_list = []

    # Iterasi melalui setiap bagian yang telah dipisahkan dengan "dan"
    for part in parts_with_and:
        # Kemudian, pisahkan setiap bagian dengan pemisah "koma"
        subparts = part.split(",")
        # Hapus spasi di awal dan akhir setiap elemen
        subparts = [subpart.strip() for subpart in subparts]
        # Tambahkan setiap elemen dalam subparts ke dalam result_list
        result_list.extend(subparts)

    # Hasil akhir adalah list yang telah dipisahkan
    return result_list

In [10]:
# test function
split_string(df_lahan.loc[4, 'Tekstur Tanah'])

['liat', 'liat berpasir', 'lempung berpasir', 'lempung bedebu']

In [11]:
def replace_with_mid_value(value):
    # parse value with comma
    if ',' in value:
        value = value.replace(',', '.')
        min_val, max_val = map(float, value.split("-"))
    else:
        # split value into 2
        min_val, max_val = map(float, value.split("-"))
    # calculate mid value
    mid_val = (min_val + max_val) / 2
    return mid_val

In [12]:
def replace_kemiringan(value):
    if value == 60:
        return value
    
    if value == '> 61':
        value = '61 - 100'

    value = value.replace('<', '').replace('>', '')

    return replace_with_mid_value(value)

**Temperature Range**

In [13]:
# Terapkan fungsi penggantian pada kolom "Suhu (°C)"
df_lahan['Suhu  '] = df_lahan['Suhu  '].apply(replace_with_mid_value)


df_lahan.head()

,No,Desa/Keluarahan,Suhu,Curah Hujan (mm/thn),Kelembapan Udara (%),Jenis Tanah,Tekstur Tanah,Ph Tanah,Kemiringan Lahan (%),Tinggi Tempat (mdpl)
0,1,Kota Uneng,23.5,622-1425,45-80,Mediteran,Lempung berpasir dan bedebu,6 - 7.5,0 - < 4,0-75
1,2,Nangalimang,23.5,622-1425,45-80,Mediteran,Lempung berpasir dan bedebu,6 - 7.5,0 - < 4,25-75
2,3,Wuring,23.5,622-1425,45-80,Mediteran,Liat berpasir dan lempung bedebu,4.2- 6,26-45,0-500
3,4,Wolomarang,23.5,622-1425,45-80,Mediteran,Liat berpasir dan lempung bedebu,4.2- 6,26-45,0-500
4,5,Tilang,28.0,1173-2277,74-86,"Regosol, Latosol dan Grumosol","Liat, Liat berpasir,Lempung berpasir dan lempu...",4.2- 6,15-39,0-1000


In [14]:
df_lahan = df_lahan.rename(columns={'Suhu  ': 'Suhu'})
df_lahan.columns

Index(['No', 'Desa/Keluarahan', 'Suhu', 'Curah Hujan (mm/thn)',
       'Kelembapan Udara (%)', 'Jenis Tanah', 'Tekstur Tanah', 'Ph Tanah',
       'Kemiringan Lahan (%)', 'Tinggi Tempat (mdpl)'],
      dtype='object')

**Rainfall Range**

In [15]:
# Terapkan fungsi penggantian pada kolom "Curah Hujan (mm/thn)"
df_lahan['Curah Hujan (mm/thn)'] = df_lahan['Curah Hujan (mm/thn)'].apply(replace_with_mid_value)

df_lahan.head()

,No,Desa/Keluarahan,Suhu,Curah Hujan (mm/thn),Kelembapan Udara (%),Jenis Tanah,Tekstur Tanah,Ph Tanah,Kemiringan Lahan (%),Tinggi Tempat (mdpl)
0,1,Kota Uneng,23.5,1023.5,45-80,Mediteran,Lempung berpasir dan bedebu,6 - 7.5,0 - < 4,0-75
1,2,Nangalimang,23.5,1023.5,45-80,Mediteran,Lempung berpasir dan bedebu,6 - 7.5,0 - < 4,25-75
2,3,Wuring,23.5,1023.5,45-80,Mediteran,Liat berpasir dan lempung bedebu,4.2- 6,26-45,0-500
3,4,Wolomarang,23.5,1023.5,45-80,Mediteran,Liat berpasir dan lempung bedebu,4.2- 6,26-45,0-500
4,5,Tilang,28.0,1725.0,74-86,"Regosol, Latosol dan Grumosol","Liat, Liat berpasir,Lempung berpasir dan lempu...",4.2- 6,15-39,0-1000


**Humidity Range**

In [16]:
# Terapkan fungsi penggantian pada kolom "Kelembapan Udara"
df_lahan['Kelembapan Udara (%)'] = df_lahan['Kelembapan Udara (%)'].apply(replace_with_mid_value)

df_lahan.head()

,No,Desa/Keluarahan,Suhu,Curah Hujan (mm/thn),Kelembapan Udara (%),Jenis Tanah,Tekstur Tanah,Ph Tanah,Kemiringan Lahan (%),Tinggi Tempat (mdpl)
0,1,Kota Uneng,23.5,1023.5,62.5,Mediteran,Lempung berpasir dan bedebu,6 - 7.5,0 - < 4,0-75
1,2,Nangalimang,23.5,1023.5,62.5,Mediteran,Lempung berpasir dan bedebu,6 - 7.5,0 - < 4,25-75
2,3,Wuring,23.5,1023.5,62.5,Mediteran,Liat berpasir dan lempung bedebu,4.2- 6,26-45,0-500
3,4,Wolomarang,23.5,1023.5,62.5,Mediteran,Liat berpasir dan lempung bedebu,4.2- 6,26-45,0-500
4,5,Tilang,28.0,1725.0,80.0,"Regosol, Latosol dan Grumosol","Liat, Liat berpasir,Lempung berpasir dan lempu...",4.2- 6,15-39,0-1000


**Jenis Tanah**

In [17]:
# Terapkan fungsi konversi pada kolom "H" (Jenis Tanah) dan simpan hasilnya di kolom "H" juga
df_lahan['Jenis Tanah'] = df_lahan['Jenis Tanah'].apply(split_string)
df_lahan.head()

,No,Desa/Keluarahan,Suhu,Curah Hujan (mm/thn),Kelembapan Udara (%),Jenis Tanah,Tekstur Tanah,Ph Tanah,Kemiringan Lahan (%),Tinggi Tempat (mdpl)
0,1,Kota Uneng,23.5,1023.5,62.5,[mediteran],Lempung berpasir dan bedebu,6 - 7.5,0 - < 4,0-75
1,2,Nangalimang,23.5,1023.5,62.5,[mediteran],Lempung berpasir dan bedebu,6 - 7.5,0 - < 4,25-75
2,3,Wuring,23.5,1023.5,62.5,[mediteran],Liat berpasir dan lempung bedebu,4.2- 6,26-45,0-500
3,4,Wolomarang,23.5,1023.5,62.5,[mediteran],Liat berpasir dan lempung bedebu,4.2- 6,26-45,0-500
4,5,Tilang,28.0,1725.0,80.0,"[regosol, latosol, grumosol]","Liat, Liat berpasir,Lempung berpasir dan lempu...",4.2- 6,15-39,0-1000


**Tekstur Tanah**

In [18]:
# Terapkan fungsi konversi pada kolom "H" (Jenis Tanah) dan simpan hasilnya di kolom "H" juga
df_lahan['Tekstur Tanah'] = df_lahan['Tekstur Tanah'].apply(split_string)
df_lahan.head()

,No,Desa/Keluarahan,Suhu,Curah Hujan (mm/thn),Kelembapan Udara (%),Jenis Tanah,Tekstur Tanah,Ph Tanah,Kemiringan Lahan (%),Tinggi Tempat (mdpl)
0,1,Kota Uneng,23.5,1023.5,62.5,[mediteran],"[lempung berpasir, bedebu]",6 - 7.5,0 - < 4,0-75
1,2,Nangalimang,23.5,1023.5,62.5,[mediteran],"[lempung berpasir, bedebu]",6 - 7.5,0 - < 4,25-75
2,3,Wuring,23.5,1023.5,62.5,[mediteran],"[liat berpasir, lempung bedebu]",4.2- 6,26-45,0-500
3,4,Wolomarang,23.5,1023.5,62.5,[mediteran],"[liat berpasir, lempung bedebu]",4.2- 6,26-45,0-500
4,5,Tilang,28.0,1725.0,80.0,"[regosol, latosol, grumosol]","[liat, liat berpasir, lempung berpasir, lempun...",4.2- 6,15-39,0-1000


**pH Range**

In [19]:
# Terapkan fungsi penggantian pada kolom "Ph Tanah"
df_lahan['Ph Tanah'] = df_lahan['Ph Tanah'].apply(replace_with_mid_value)

df_lahan.head()

,No,Desa/Keluarahan,Suhu,Curah Hujan (mm/thn),Kelembapan Udara (%),Jenis Tanah,Tekstur Tanah,Ph Tanah,Kemiringan Lahan (%),Tinggi Tempat (mdpl)
0,1,Kota Uneng,23.5,1023.5,62.5,[mediteran],"[lempung berpasir, bedebu]",6.75,0 - < 4,0-75
1,2,Nangalimang,23.5,1023.5,62.5,[mediteran],"[lempung berpasir, bedebu]",6.75,0 - < 4,25-75
2,3,Wuring,23.5,1023.5,62.5,[mediteran],"[liat berpasir, lempung bedebu]",5.10,26-45,0-500
3,4,Wolomarang,23.5,1023.5,62.5,[mediteran],"[liat berpasir, lempung bedebu]",5.10,26-45,0-500
4,5,Tilang,28.0,1725.0,80.0,"[regosol, latosol, grumosol]","[liat, liat berpasir, lempung berpasir, lempun...",5.10,15-39,0-1000


**Kemiringan Lahan**

In [20]:
# Terapkan fungsi penggantian pada kolom "Kemiringan Lahan"
df_lahan['Kemiringan Lahan (%)'] = df_lahan['Kemiringan Lahan (%)'].apply(replace_kemiringan)

df_lahan.head()

,No,Desa/Keluarahan,Suhu,Curah Hujan (mm/thn),Kelembapan Udara (%),Jenis Tanah,Tekstur Tanah,Ph Tanah,Kemiringan Lahan (%),Tinggi Tempat (mdpl)
0,1,Kota Uneng,23.5,1023.5,62.5,[mediteran],"[lempung berpasir, bedebu]",6.75,2.0,0-75
1,2,Nangalimang,23.5,1023.5,62.5,[mediteran],"[lempung berpasir, bedebu]",6.75,2.0,25-75
2,3,Wuring,23.5,1023.5,62.5,[mediteran],"[liat berpasir, lempung bedebu]",5.10,35.5,0-500
3,4,Wolomarang,23.5,1023.5,62.5,[mediteran],"[liat berpasir, lempung bedebu]",5.10,35.5,0-500
4,5,Tilang,28.0,1725.0,80.0,"[regosol, latosol, grumosol]","[liat, liat berpasir, lempung berpasir, lempun...",5.10,27.0,0-1000


**Tinggi Tempat Range**

In [21]:
# Terapkan fungsi penggantian pada kolom "Tinggi Tempat"
df_lahan['Tinggi Tempat (mdpl)'] = df_lahan['Tinggi Tempat (mdpl)'].apply(replace_with_mid_value)

df_lahan.head()

,No,Desa/Keluarahan,Suhu,Curah Hujan (mm/thn),Kelembapan Udara (%),Jenis Tanah,Tekstur Tanah,Ph Tanah,Kemiringan Lahan (%),Tinggi Tempat (mdpl)
0,1,Kota Uneng,23.5,1023.5,62.5,[mediteran],"[lempung berpasir, bedebu]",6.75,2.0,37.5
1,2,Nangalimang,23.5,1023.5,62.5,[mediteran],"[lempung berpasir, bedebu]",6.75,2.0,50.0
2,3,Wuring,23.5,1023.5,62.5,[mediteran],"[liat berpasir, lempung bedebu]",5.10,35.5,250.0
3,4,Wolomarang,23.5,1023.5,62.5,[mediteran],"[liat berpasir, lempung bedebu]",5.10,35.5,250.0
4,5,Tilang,28.0,1725.0,80.0,"[regosol, latosol, grumosol]","[liat, liat berpasir, lempung berpasir, lempun...",5.10,27.0,500.0


### Export Preprocessed Data

In [22]:
df_lahan.head()

,No,Desa/Keluarahan,Suhu,Curah Hujan (mm/thn),Kelembapan Udara (%),Jenis Tanah,Tekstur Tanah,Ph Tanah,Kemiringan Lahan (%),Tinggi Tempat (mdpl)
0,1,Kota Uneng,23.5,1023.5,62.5,[mediteran],"[lempung berpasir, bedebu]",6.75,2.0,37.5
1,2,Nangalimang,23.5,1023.5,62.5,[mediteran],"[lempung berpasir, bedebu]",6.75,2.0,50.0
2,3,Wuring,23.5,1023.5,62.5,[mediteran],"[liat berpasir, lempung bedebu]",5.10,35.5,250.0
3,4,Wolomarang,23.5,1023.5,62.5,[mediteran],"[liat berpasir, lempung bedebu]",5.10,35.5,250.0
4,5,Tilang,28.0,1725.0,80.0,"[regosol, latosol, grumosol]","[liat, liat berpasir, lempung berpasir, lempun...",5.10,27.0,500.0


In [24]:
df_lahan.to_excel('../data/Data Lahan Processed.xlsx', index=False)  # index=False untuk mengabaikan penambahan index

# Tampilkan pesan setelah berhasil menyimpan
print("Data berhasil diekspor ke Excel.")

Data berhasil diekspor ke Excel.


### Interpolation